<center>
  <h2>Trabajo Laboratorio de Implementación III - MCD Virtual Cohorte 2022</h2>
  <h3>Forecasting para una Empresa de Consumo Masivo</h3>
  <h4>Armado de Datasets</h4>
</center>

In [260]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
warnings.filterwarnings("ignore")

date_start=datetime.strptime('2017-01-01', '%Y-%m-%d')
date_end=datetime.strptime('2019-12-01', '%Y-%m-%d')
forecast_length = 2
date_future=date_end+relativedelta(months=forecast_length)
years=range(2018, 2020)

In [261]:
print(f"Fecha inicio historia {date_start}")
print(f"Fecha fin historia {date_end}")
print(f"Fecha futura (related time series) {date_future}")

Fecha inicio historia 2017-01-01 00:00:00
Fecha fin historia 2019-12-01 00:00:00
Fecha futura (related time series) 2020-02-01 00:00:00


- Skus a predecir

In [262]:
skus_a_predecir = pd.read_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Datasets/productos_a_predecir.txt", delimiter='\t')
skus_a_predecir = skus_a_predecir.product_id.to_list()

- Datos de ventas

In [263]:
df_ventas = pd.read_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Datasets/sell-in.txt", delimiter='\t')
df_ventas['periodo'] = df_ventas['periodo'].astype(str)
df_ventas['periodo'] = pd.to_datetime(df_ventas['periodo'], format='%Y%m')
df_ventas = df_ventas[df_ventas.product_id.isin(skus_a_predecir)]
df_ventas.head(1)

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,2017-01-01,10234,20524,0,2,0.053,0.053


In [264]:
print(f"Cantidad productos finales a predecir: {len(df_ventas.product_id.drop_duplicates())}")

Cantidad productos finales a predecir: 780


Dataset de ventas para predecir unicamente por periodo y producto:

In [265]:
df_ventas_product = df_ventas.groupby(["periodo", "product_id"])["tn"].sum().reset_index()
df_ventas_product.head(1)

,periodo,product_id,tn
0,2017-01-01,20001,934.77222


In [266]:
df_ventas_product.isna().sum()

periodo       0
product_id    0
tn            0
dtype: int64

In [267]:
df_ventas_product.tn.sum().round(1) == df_ventas.tn.sum().round(1) 

True

- Cortamos outliers

In [268]:
# FUNCIÓN PARA IDENTIFICAR OUTLIERS MÁXIMOS Y LLEVAR LOS VALORES AL P95 DE LA DISTRIBUCION
def fill_sales_extreme_upper_values_by_percentile_method(df, upper_quantile=0.95, grouped_columns=[]):
    '''Calculates sales extreme values for those that are greater than an upper percentile
    and fills upper outliers with the upper percentile value.
    '''
    def upperextreme(x):
        '''Define Upper Extreme'''
        return x.quantile(q=upper_quantile)
    
    # Calculate upper extreme values by grouped_columns
    df_upper_extremes = df.groupby(grouped_columns)["tn"].agg([upperextreme]).reset_index()
    # Merge the upper extreme values back to the original dataframe
    df = df.merge(df_upper_extremes, on=grouped_columns)
    # Replace upper extreme values with upper quantile value
    df['tn'] = np.where(df['tn'] > df['upperextreme'], df['upperextreme'], df['tn'])
    # Drop the upper extreme value columns as they are no longer needed
    df = df.drop(columns=['upperextreme'])
    return df

# Aplicación de la función
df_ventas_product_cut_upper = fill_sales_extreme_upper_values_by_percentile_method(df_ventas_product, 
                                                                                   upper_quantile=0.95, 
                                                                                   grouped_columns=["product_id"])

In [269]:
# Exportamos data de ventas product ACOTADA por outliers EXTREMOS INFERIORES Y SUPERIORES
df_ventas_product_export = df_ventas_product_cut_upper[(df_ventas_product_cut_upper['periodo']>=date_start) & (df_ventas_product_cut_upper['periodo']<=date_end)]
df_ventas_product_export['product_id'] = df_ventas_product_export['product_id'].astype(str)
print(f"Min periodo {df_ventas_product_export.periodo.min()}")
print(f"Max periodo {df_ventas_product_export.periodo.max()}")
df_ventas_product_export.to_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Datasets/df_ventas_product_fill_extreme_values_095.csv", index=False, header=False)
print(f"Tipos por columna: \n{df_ventas_product_export.dtypes}")

Min periodo 2017-01-01 00:00:00
Max periodo 2019-12-01 00:00:00
Tipos por columna: 
periodo       datetime64[ns]
product_id            object
tn                   float64
dtype: object


- Datos de producto

In [270]:
producto = pd.read_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Datasets/tb_productos.txt", delimiter='\t')
producto.head(5)

,cat1,cat2,cat3,brand,sku_size,product_id
0,HC,ROPA LAVADO,Liquido,LIMPIEX,900,20280
1,HC,ROPA LAVADO,Liquido,LIMPIEX,450,20180
2,HC,ROPA LAVADO,Liquido,LIMPIEX,120,20332
3,HC,ROPA LAVADO,Liquido,LIMPIEX,450,20222
4,HC,ROPA LAVADO,Liquido,LIMPIEX,900,20288


In [271]:
print("----ANALISIS DUPLICADOS-----")
print(f"Cantidad productos en maestro productos: {len(producto)}")
print(f"Cantidad productos en maestro productos sin repetidos: {len(producto.product_id.drop_duplicates())}")
# Observamos que product_id duplicados tienen los mismos valores en las otras columnas, son duplicados y procedemos a eliminarlos 
producto = producto.drop_duplicates(subset = "product_id")
print(f"Cantidad productos en maestro productos: {len(producto)}")

----ANALISIS DUPLICADOS-----
Cantidad productos en maestro productos: 1262
Cantidad productos en maestro productos sin repetidos: 1251
Cantidad productos en maestro productos: 1251


In [272]:
producto = producto[producto.product_id.isin(skus_a_predecir)]
print(f"Cantidad productos finales a predecir: {len(producto)}")

Cantidad productos finales a predecir: 780


- Correlaciones de cada serie con productos top 10

Los 10% más importantes contienen el 20% de las tn totales, analizamos la correlación de cada serie con ese top 10

In [273]:
df_ventas_product_2019 = df_ventas_product[df_ventas_product.periodo.dt.year == 2019]
df_ventas_product_2019_grouped = df_ventas_product_2019.groupby(["product_id"])["tn"].sum()
top_10_products = (df_ventas_product_2019_grouped / df_ventas_product_2019.tn.sum()) * 100
top_10_products = top_10_products.reset_index().sort_values(by = "tn", ascending = False).head(10)
top_10_products = top_10_products["product_id"].to_list()

In [274]:
# Filtrar y pivotear los datos
pivot_df = df_ventas_product_2019.pivot(index='periodo', columns='product_id', values='tn')
# Crear un DataFrame para almacenar las correlaciones
correlation_results = pd.DataFrame(index=pivot_df.columns)
# Calcular la correlación de cada producto con cada uno de los top 10 productos
for top_product in top_10_products:
    correlations = pivot_df.corrwith(pivot_df[top_product])
    correlation_results["corr_" + str(top_product)] = correlations
correlation_results = correlation_results.reset_index()
correlation_results

,product_id,corr_20001,corr_20002,corr_20003,corr_20005,corr_20004,corr_20009,corr_20032,corr_20006,corr_20007,corr_20010
0,20001,1.000000,0.292781,0.040044,0.418139,0.235897,0.164754,0.016506,0.235667,0.226113,0.174968
1,20002,0.292781,1.000000,0.654610,0.307409,0.720955,0.067080,0.328798,0.360105,0.030957,0.185156
2,20003,0.040044,0.654610,1.000000,0.124172,0.664815,0.152551,0.483269,-0.168530,-0.378631,0.058513
3,20004,0.235897,0.720955,0.664815,0.692485,1.000000,0.372809,0.556129,-0.084853,0.003766,0.419989
4,20005,0.418139,0.307409,0.124172,1.000000,0.692485,0.479811,0.341879,-0.044414,0.203697,0.488491
...,...,...,...,...,...,...,...,...,...,...,...
775,21263,-0.265714,-0.177419,-0.197963,-0.734870,-0.453351,-0.241550,-0.224398,-0.059761,0.077890,-0.236078
776,21265,0.366056,0.272910,-0.342837,0.079827,-0.207902,-0.540198,-0.608804,0.901903,0.135442,-0.004080
777,21266,0.417087,0.289371,-0.328367,0.111039,-0.202629,-0.503188,-0.597295,0.895020,0.171336,0.015036
778,21267,0.267666,0.014517,-0.528666,0.033034,-0.136412,-0.169159,-0.428888,0.450743,0.382757,0.134982


In [275]:
len(producto)

780

In [276]:
producto = pd.merge(producto, correlation_results, how = "inner", on = "product_id")
producto.head(1)

,cat1,cat2,cat3,brand,sku_size,product_id,corr_20001,corr_20002,corr_20003,corr_20005,corr_20004,corr_20009,corr_20032,corr_20006,corr_20007,corr_20010
0,HC,ROPA LAVADO,Liquido,LIMPIEX,900,20280,0.344376,0.251042,0.149971,0.056807,0.179661,0.107151,0.091045,-0.142871,0.511455,0.054308


In [277]:
len(producto)

780

- Exportamos data

In [278]:
producto = producto[["product_id", "cat1", "cat2", "cat3", "brand", "sku_size"] + producto.filter(like='corr_').columns.tolist()]
producto['product_id'] = producto['product_id'].astype(str)
producto.to_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Datasets/df_products_corr.csv", index=False, header=False)
print(f"Tipos por columna \n{producto.dtypes}")

Tipos por columna 
product_id     object
cat1           object
cat2           object
cat3           object
brand          object
sku_size        int64
corr_20001    float64
corr_20002    float64
corr_20003    float64
corr_20005    float64
corr_20004    float64
corr_20009    float64
corr_20032    float64
corr_20006    float64
corr_20007    float64
corr_20010    float64
dtype: object


- Creación serie de tiempo relacionada

Agregamos fechas faltantes para tener todos los puntos de historia y no sólo donde tenemos venta registrada

In [279]:
def create_future_periods(df):
    combination = df.groupby([i for i in ['product_id']])["periodo"].apply(lambda s: pd.date_range(s.min(), date_future, freq='MS')).explode().reset_index()
    result = pd.merge(combination, df, how = "left", on = ["periodo", "product_id"])
    result = result[["periodo", "product_id", "tn"]]
    return result

In [280]:
df_related_product_v1 = create_future_periods(df_ventas_product_cut_upper)

Agregamos variable relativas a tiempo

In [281]:
def time_features(df):
    df_time_features = df.copy()
    # Asegúrate de que la columna 'periodo' esté en formato datetime
    df_time_features['periodo'] = pd.to_datetime(df_time_features['periodo'], format='%Y-%m-%d')
    # Crear las columnas 'mes', 'cuatrimestre' y 'dias del mes"
    df_time_features['month'] = df_time_features['periodo'].dt.month
    df_time_features['quarter'] = df_time_features['periodo'].dt.quarter
    df_time_features['days_in_month']= df_time_features['periodo'].dt.daysinmonth
    return df_time_features

In [282]:
df_related_product_v2 = time_features(df_related_product_v1)

Agregamos cantidad de sabados y domingos del mes

In [283]:
# Cantidad de domingos al mes
sundays = pd.DataFrame(pd.date_range(start=pd.to_datetime(date_start), end=date_future + pd.offsets.MonthEnd(0)))
# sundays = pd.DataFrame(pd.date_range(start=pd.to_datetime(date_start) - pd.DateOffset(years=1), end=date_future + pd.offsets.MonthEnd(0)))
sundays.columns = ["fecha"]
sundays["day_week"] = pd.to_datetime(sundays['fecha'], format='%Y-%m-%d').dt.dayofweek
sundays = sundays[sundays.day_week == 6].rename(columns = {"day_week":"sundays"})
sundays = sundays.replace({6:1})
sundays['periodo']=sundays['fecha'].apply(lambda x: x.replace(day=1))
sundays = sundays.groupby(["periodo"])["sundays"].sum().reset_index()
sundays['periodo']=pd.to_datetime(sundays['periodo'], format='%Y-%m-%d')
sundays["sundays"] = sundays["sundays"].astype(int)
# Cantidad de sabados al mes
saturdays = pd.DataFrame(pd.date_range(start=pd.to_datetime(date_start), end=date_future + pd.offsets.MonthEnd(0)))
# saturdays = pd.DataFrame(pd.date_range(start=pd.to_datetime(date_start) - pd.DateOffset(years=1), end=date_future + pd.offsets.MonthEnd(0)))
saturdays.columns = ["fecha"]
saturdays["day_week"] = pd.to_datetime(saturdays['fecha'], format='%Y-%m-%d').dt.dayofweek
saturdays = saturdays[saturdays.day_week == 5].rename(columns = {"day_week":"saturdays"})
saturdays = saturdays.replace({5:1})
saturdays['periodo']=saturdays['fecha'].apply(lambda x: x.replace(day=1))
saturdays = saturdays.groupby(["periodo"])["saturdays"].sum().reset_index()
saturdays["saturdays"] = saturdays["saturdays"].astype(int)
saturdays['periodo']=pd.to_datetime(saturdays['periodo'], format='%Y-%m-%d')
# DF conjunto
sundays_saturdays = pd.merge(sundays, saturdays, how = "inner", on = "periodo")
sundays_saturdays.head(5)

,periodo,sundays,saturdays
0,2017-01-01,5,4
1,2017-02-01,4,4
2,2017-03-01,4,4
3,2017-04-01,5,5
4,2017-05-01,4,4


In [284]:
df_related_product_v3 = pd.merge(df_related_product_v2, sundays_saturdays, how = "left", on = "periodo")

- Agregamos desfasajes y lags de los últimos 12 meses

In [285]:
def shift_series(df, columns_shift):
    df_shift = df.copy()
    df_shift = df_shift.sort_values(by=columns_shift + ["periodo"])
    lags = range(1, 13)
    for lag in lags:
        df_shift[f'lag{lag}m'] = df_shift.groupby(columns_shift)['tn'].shift(lag)
        deltas = range(1, 13) 
    for delta in deltas:
        df_shift[f'delta{delta}'] = df_shift.groupby(columns_shift)['tn'].diff(periods=delta)
    return df_shift

In [286]:
df_related_product_v4 = shift_series(df_related_product_v3, columns_shift = ["product_id"])
# Los puntos de tn con nan durante el periodo de train, son rellenados con cero, para el futuro se quedan con nan
df_related_product_v4["tn"] = np.where(df_related_product_v4["periodo"] <= date_end, df_related_product_v4["tn"].fillna(0), df_related_product_v4["tn"])
df_related_product_v4[df_related_product_v4.periodo > date_end].tn.unique()

array([nan])

Agregamos feature de % de ventas mensual sobre el total anual (por product_id) para recoger estacionalidad

In [287]:
def calculate_monthly_percentage(df, columns_shift):
    df_pesos = df.copy()
    df_pesos["year"] = df_pesos.periodo.dt.year
    # Calcular el volumen total anual por product_id
    annual_volume = df_pesos.groupby(columns_shift + ['year'])['tn'].sum().reset_index()
    annual_volume.rename(columns={'tn': 'annual_tn'}, inplace=True)
    # Unir el volumen anual de vuelta al DataFrame original
    df_pesos = pd.merge(df_pesos, annual_volume, on=columns_shift + ['year'], how='left')
    # Calcular el porcentaje del volumen mensual respecto al volumen anual (por año)
    df_pesos['monthly_percentage'] = df_pesos['tn'] / df_pesos['annual_tn'] * 100
    # Crear una columna shifted_monthly_percentage con el valor de monthly_percentage hace 12 meses
    df_pesos['shifted_monthly_percentage'] = df_pesos.groupby(columns_shift)['monthly_percentage'].shift(12)
    # Rellenar los NaN en monthly_percentage con los valores de shifted_monthly_percentage (serían los valores futuros)
    df_pesos['monthly_percentage'] = df_pesos['monthly_percentage'].fillna(df_pesos['shifted_monthly_percentage'])
    # Eliminar las columnas tn, shifted_monthly_percentage y year si no son necesarias
    df_pesos = df_pesos.drop(columns=['shifted_monthly_percentage', 'year', 'annual_tn'])
    df_pesos['monthly_percentage'] = df_pesos['monthly_percentage'].fillna(0)
    return df_pesos

In [288]:
df_related_product_v5 = calculate_monthly_percentage(df_related_product_v4, columns_shift = ["product_id"])
df_related_product_v5 = df_related_product_v5.drop(columns = "tn")

In [289]:
len(df_related_product_v5) == len(df_related_product_v1)

True

- Agregamos series macro relacionadas

In [290]:
df_ipc = pd.read_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Datasets/BCRA_inflacion_esperada_proximos_12_meses.csv", sep=';')
df_ipc = df_ipc.rename(columns = {"Fecha":"periodo", "inflacion_esperada_12_meses": "ipc"})
df_ipc['periodo'] = pd.to_datetime(df_ipc['periodo'], format='%d/%m/%Y').dt.to_period('M').dt.to_timestamp()
df_ipc.dtypes

periodo    datetime64[ns]
ipc               float64
dtype: object

In [291]:
import requests
import json
fecha_desde = "2017-01-01"
fecha_hasta = "2020-02-29"
url = f"https://mercados.ambito.com//dolar/informal/grafico/{fecha_desde}/{fecha_hasta}"
response = requests.get(url)
data = json.loads(response.text)
df_dolar = pd.DataFrame(data)
df_dolar.columns = ["periodo", "dolar"]
df_dolar = df_dolar.iloc[1:, :]
df_dolar['periodo'] = pd.to_datetime(df_dolar['periodo'], format='%d/%m/%Y').dt.to_period('M').dt.to_timestamp()
df_dolar = df_dolar.groupby(["periodo"])["dolar"].mean().reset_index()
df_dolar["dolar"] = df_dolar["dolar"].astype(float)
df_dolar.dtypes

periodo    datetime64[ns]
dolar             float64
dtype: object

In [292]:
df_macro = pd.merge(df_ipc, df_dolar, how ="inner", on = "periodo")
df_macro.head(1)

,periodo,ipc,dolar
0,2017-01-01,19.7,16.853182


In [293]:
df_macro.periodo.min(), df_macro.periodo.max()

(Timestamp('2017-01-01 00:00:00'), Timestamp('2020-02-01 00:00:00'))

In [294]:
df_related_product_v6 = pd.merge(df_related_product_v5, df_macro, how ="left", on = "periodo")
df_related_product_v6.head(5)

,periodo,product_id,month,quarter,days_in_month,sundays,saturdays,lag1m,lag2m,lag3m,...,delta6,delta7,delta8,delta9,delta10,delta11,delta12,monthly_percentage,ipc,dolar
0,2017-01-01,20001,1,1,31,5,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.310483,19.7,16.853182
1,2017-02-01,20001,2,1,28,4,4,934.77222,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.387267,19.5,16.425000
2,2017-03-01,20001,3,1,31,4,4,798.01620,934.77222,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.798738,18.4,16.062273
3,2017-04-01,20001,4,2,30,5,5,1303.35771,798.01620,934.77222,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.223120,17.5,15.921667
4,2017-05-01,20001,5,2,31,4,4,1069.96130,1303.35771,798.01620,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.141096,17.0,16.104762


- Exportamos

In [295]:
# Exportamos data relacionada, apertura por product
# Nos aseguramos que cada serie temporal (por producto) comience en el mismo periodo que comienza en la serie de tiempo ppal de ventas
df_min_period = df_ventas_product_export.groupby(["product_id"])["periodo"].min().reset_index().rename(columns = {"periodo":"min_periodo"})
df_related_product_v6['product_id'] = df_related_product_v6['product_id'].astype(str)
df_related_product_export = pd.merge(df_related_product_v6, df_min_period, on='product_id')
df_related_product_export = df_related_product_export[df_related_product_export['periodo'] >= df_related_product_export['min_periodo']]
df_related_product_export = df_related_product_export.drop(columns=['min_periodo'])
df_related_product_export = df_related_product_export[(df_related_product_export['periodo']<=date_future)]
df_related_product_export.to_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Datasets/df_related_product_v2.csv", index=False, header=False)
print("-------------------------")
print(f"Min periodo {df_related_product_export.periodo.min()}")
print(f"Max periodo {df_related_product_export.periodo.max()}")
print(f"Cantidad de productos {len(df_related_product_export.product_id.unique())}")
print("-------------------------")
print(f"Tipos por columna \n{df_related_product_export.dtypes}")
print("-------------------------")
print(f"Cantidad de NAN values \n{df_related_product_export.isna().sum()}")
print("-------------------------")

-------------------------
Min periodo 2017-01-01 00:00:00
Max periodo 2020-02-01 00:00:00
Cantidad de productos 780
-------------------------
Tipos por columna 
periodo               datetime64[ns]
product_id                    object
month                          int32
quarter                        int32
days_in_month                  int32
sundays                        int32
saturdays                      int32
lag1m                        float64
lag2m                        float64
lag3m                        float64
lag4m                        float64
lag5m                        float64
lag6m                        float64
lag7m                        float64
lag8m                        float64
lag9m                        float64
lag10m                       float64
lag11m                       float64
lag12m                       float64
delta1                       float64
delta2                       float64
delta3                       float64
delta4                   

In [296]:
df_related_product_export.columns.to_list()

['periodo',
 'product_id',
 'month',
 'quarter',
 'days_in_month',
 'sundays',
 'saturdays',
 'lag1m',
 'lag2m',
 'lag3m',
 'lag4m',
 'lag5m',
 'lag6m',
 'lag7m',
 'lag8m',
 'lag9m',
 'lag10m',
 'lag11m',
 'lag12m',
 'delta1',
 'delta2',
 'delta3',
 'delta4',
 'delta5',
 'delta6',
 'delta7',
 'delta8',
 'delta9',
 'delta10',
 'delta11',
 'delta12',
 'monthly_percentage',
 'ipc',
 'dolar']

Chequear que cada combinacion en el df relacionado comience en misma fecha que en df principal

In [297]:
fecha_min_ventas = df_ventas_product_export.groupby(["product_id"])["periodo"].min().reset_index().rename(columns = {"periodo":"min_period_ventas"})
fecha_min_related = df_related_product_export.groupby(["product_id"])["periodo"].min().reset_index().rename(columns = {"periodo":"min_period_related"})
merge_fecha_min = pd.merge(fecha_min_ventas, fecha_min_related, how = "outer", on = "product_id")
merge_fecha_min[merge_fecha_min.min_period_ventas != merge_fecha_min.min_period_related]

,product_id,min_period_ventas,min_period_related
